In [1]:
import os
import glob
import random
import time
from utils import mfccs_and_spec
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from random import shuffle
from hparam import hparam as hp
from torch.functional import F
import torch.nn as nn
from torch.utils.data import Dataset
from torch.autograd import Variable, Function
import torch.optim as optim
from data_load import TripletSpeakerDataset
from speech_embedder_net import SpeechEmbedder, GE2ELoss, get_centroids, get_cossim

/home/dexter/Desktop/projects/Voice-Cloning/hparam.py:11: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


In [2]:
import librosa 
import librosa.display
import IPython
import pickle 
import numpy as np
import scipy 
import tensorflow as tf 
from sklearn.externals import joblib
from tensorflow.python.client import device_lib
from keras.layers import Dense, Dropout, BatchNormalization
from keras.models import Sequential
from keras import backend as K
from keras.optimizers import Adam
from tensorflow.saved_model import simple_save

Using TensorFlow backend.


In [3]:
from utils import mfcc_for_accent
from data_preprocess import get_spectrogram_tisv

In [4]:
file_name = "/home/dexter/Desktop/projects/Mini/data/VCTK-Corpus/wav48_silence_trimmed/p225/p225_001_mic2.flac"

In [4]:
config = tf.ConfigProto(allow_soft_placement = True,
                        device_count = {'CPU' : 6,
                                       'GPU' : 0})

In [5]:
session = tf.Session(config=config)
K.set_session(session)

In [6]:
def make_accent_model():
    model = Sequential()

    model.add(Dense(50, input_shape = (14976,), activation = 'relu',name='input'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(75, activation = 'tanh', name='h1'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(100,  activation = 'tanh',name = 'h2'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(100,  activation = 'tanh', name = 'h3'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(100,  activation = 'tanh', name = 'h4'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(100,  activation = 'tanh', name = 'h5'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(100,  activation = 'tanh', name ='h6'))
    model.add(BatchNormalization())

    model.add(Dense(3,  activation = 'softmax', name = 'output'))
    return model

In [7]:
##Mel spectrograms and MFCCs for Indians 
def get_accent_mfccs(path):
    x, _ = librosa.load(path)
    x, mf = mfcc_for_accent(x)
    mf = mf.flatten()
    #print(type(x))
    # l.append(librosa.feature.mfcc(x, sr=f))
    
    dat = np.reshape(mf, (1,mf.shape[0]))
    return x, dat

In [8]:

embedder_net = SpeechEmbedder().to('cuda')
embedder_net.load_state_dict(torch.load(hp.model.model_path))
accent_net = make_accent_model()
accent_net.load_weights('accent_block_with_scottish.h5')
accent_net.pop()


#wav, sr = librosa.load(file_name)

def run_speaker_encoder(mfccs, embedder_net):
    mel_db = torch.transpose(mfccs, 1,2)
    #mel_db = torch.from_numpy(mel_db)
    #print(mel_db.shape)
    out = embedder_net(mel_db.to('cuda'))
    return out

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [17]:
def get_both_embs(path):
        #for path in enumerate(train_dataset):
        _, accent_mfccs = get_accent_mfccs(path)
        speaker_mfccs = get_spectrogram_tisv(path)
        #print(accent_mfccs.shape)
        print(speaker_mfccs.shape)
        speaker_mfccs = Variable(torch.from_numpy(speaker_mfccs)).to('cuda')
        speaker_emb = run_speaker_encoder(speaker_mfccs, embedder_net)
        #print(speaker_emb.shape)
        accent_emb = accent_net.predict(accent_mfccs)
        #print(accent_emb.shape)
        return speaker_emb.cpu().detach().numpy(), accent_emb

In [18]:
sp, ac = get_both_embs(file_name)

NameError: name 'file_name' is not defined

In [19]:
dataset = TripletSpeakerDataset()
data_loader = torch.utils.data.DataLoader(dataset, drop_last=True)

In [20]:
for batch_idx, (anchor_utter_path, positive_utter_path, negative_utter_path, accent_negative_path, speaker_positive_id, speaker_negative_id, speaker_anchor_id, positive_accent, negative_accent) in enumerate(data_loader):
    print(anchor_utter_path, positive_utter_path, negative_utter_path, accent_negative_path, speaker_positive_id, speaker_negative_id, speaker_anchor_id, positive_accent, negative_accent)

('data/Irish/p283/p283_040_mic2.flac',) ('data/Irish/p283/p283_396_mic1.flac',) ('data/Irish/p298/p298_324_mic1.flac',) ('data/Scottish/p237/p237_290_mic2.flac',) ('p283',) ('p298',) ('p283',) ('Irish',) ('Scottish',)
('data/American/p299/p299_110_mic1.flac',) ('data/American/p299/p299_347_mic1.flac',) ('data/American/p300/p300_313_mic1.flac',) ('data/Scottish/p284/p284_324_mic1.flac',) ('p299',) ('p300',) ('p299',) ('American',) ('Scottish',)
('data/Scottish/p284/p284_039_mic2.flac',) ('data/Scottish/p284/p284_386_mic2.flac',) ('data/Scottish/p237/p237_202_mic1.flac',) ('data/American/p301/p301_098_mic2.flac',) ('p284',) ('p237',) ('p284',) ('Scottish',) ('American',)
('data/Scottish/p237/p237_292_mic2.flac',) ('data/Scottish/p237/p237_191_mic2.flac',) ('data/Scottish/p249/p249_178_mic2.flac',) ('data/Irish/p245/p245_119_mic2.flac',) ('p237',) ('p249',) ('p237',) ('Scottish',) ('Irish',)
('data/American/p299/p299_302_mic1.flac',) ('data/American/p299/p299_151_mic2.flac',) ('data/Ameri

('data/American/p299/p299_323_mic1.flac',) ('data/American/p299/p299_081_mic1.flac',) ('data/American/p301/p301_254_mic2.flac',) ('data/Indian/p376/p376_298_mic1.flac',) ('p299',) ('p301',) ('p299',) ('American',) ('Indian',)
('data/Scottish/p237/p237_183_mic1.flac',) ('data/Scottish/p237/p237_232_mic1.flac',) ('data/Scottish/p249/p249_048_mic1.flac',) ('data/Irish/p298/p298_155_mic2.flac',) ('p237',) ('p249',) ('p237',) ('Scottish',) ('Irish',)
('data/Scottish/p284/p284_008_mic2.flac',) ('data/Scottish/p284/p284_077_mic2.flac',) ('data/Scottish/p234/p234_165_mic1.flac',) ('data/American/p301/p301_282_mic1.flac',) ('p284',) ('p234',) ('p284',) ('Scottish',) ('American',)
('data/Scottish/p237/p237_077_mic2.flac',) ('data/Scottish/p237/p237_007_mic1.flac',) ('data/Scottish/p234/p234_028_mic1.flac',) ('data/American/p299/p299_071_mic2.flac',) ('p237',) ('p234',) ('p237',) ('Scottish',) ('American',)
('data/Indian/p248/p248_235_mic2.flac',) ('data/Indian/p248/p248_070_mic2.flac',) ('data/I

('data/Indian/p376/p376_048_mic1.flac',) ('data/Indian/p376/p376_338_mic2.flac',) ('data/Indian/p248/p248_281_mic1.flac',) ('data/Irish/p298/p298_390_mic1.flac',) ('p376',) ('p248',) ('p376',) ('Indian',) ('Irish',)
('data/American/p300/p300_267_mic1.flac',) ('data/American/p300/p300_337_mic1.flac',) ('data/American/p299/p299_077_mic1.flac',) ('data/Scottish/p284/p284_183_mic1.flac',) ('p300',) ('p299',) ('p300',) ('American',) ('Scottish',)
('data/Scottish/p234/p234_081_mic1.flac',) ('data/Scottish/p234/p234_100_mic2.flac',) ('data/Scottish/p249/p249_067_mic1.flac',) ('data/Irish/p283/p283_227_mic2.flac',) ('p234',) ('p249',) ('p234',) ('Scottish',) ('Irish',)
('data/Scottish/p284/p284_309_mic1.flac',) ('data/Scottish/p284/p284_376_mic2.flac',) ('data/Scottish/p237/p237_111_mic2.flac',) ('data/Indian/p251/p251_282_mic1.flac',) ('p284',) ('p237',) ('p284',) ('Scottish',) ('Indian',)
('data/American/p301/p301_232_mic2.flac',) ('data/American/p301/p301_137_mic1.flac',) ('data/American/p2

('data/Indian/p248/p248_059_mic2.flac',) ('data/Indian/p248/p248_371_mic1.flac',) ('data/Indian/p376/p376_140_mic2.flac',) ('data/Irish/p283/p283_389_mic1.flac',) ('p248',) ('p376',) ('p248',) ('Indian',) ('Irish',)
('data/Irish/p298/p298_172_mic1.flac',) ('data/Irish/p298/p298_293_mic2.flac',) ('data/Irish/p245/p245_248_mic2.flac',) ('data/Indian/p251/p251_292_mic2.flac',) ('p298',) ('p245',) ('p298',) ('Irish',) ('Indian',)
('data/Indian/p251/p251_069_mic1.flac',) ('data/Indian/p251/p251_128_mic1.flac',) ('data/Indian/p376/p376_221_mic1.flac',) ('data/American/p301/p301_380_mic1.flac',) ('p251',) ('p376',) ('p251',) ('Indian',) ('American',)
('data/American/p301/p301_301_mic1.flac',) ('data/American/p301/p301_282_mic1.flac',) ('data/American/p300/p300_063_mic1.flac',) ('data/Indian/p376/p376_397_mic1.flac',) ('p301',) ('p300',) ('p301',) ('American',) ('Indian',)
('data/American/p299/p299_321_mic1.flac',) ('data/American/p299/p299_304_mic2.flac',) ('data/American/p300/p300_083_mic2.f

('data/Indian/p248/p248_061_mic2.flac',) ('data/Indian/p248/p248_342_mic2.flac',) ('data/Indian/p251/p251_189_mic2.flac',) ('data/Irish/p283/p283_410_mic1.flac',) ('p248',) ('p251',) ('p248',) ('Indian',) ('Irish',)
('data/Scottish/p284/p284_154_mic2.flac',) ('data/Scottish/p284/p284_349_mic1.flac',) ('data/Scottish/p237/p237_011_mic2.flac',) ('data/American/p299/p299_317_mic1.flac',) ('p284',) ('p237',) ('p284',) ('Scottish',) ('American',)
('data/Irish/p245/p245_051_mic2.flac',) ('data/Irish/p245/p245_306_mic2.flac',) ('data/Irish/p298/p298_312_mic1.flac',) ('data/Indian/p248/p248_220_mic1.flac',) ('p245',) ('p298',) ('p245',) ('Irish',) ('Indian',)
('data/Scottish/p249/p249_280_mic1.flac',) ('data/Scottish/p249/p249_070_mic2.flac',) ('data/Scottish/p237/p237_235_mic2.flac',) ('data/Irish/p298/p298_400_mic2.flac',) ('p249',) ('p237',) ('p249',) ('Scottish',) ('Irish',)
('data/Indian/p251/p251_058_mic2.flac',) ('data/Indian/p251/p251_309_mic1.flac',) ('data/Indian/p248/p248_164_mic1.f

...

('data/American/p300/p300_055_mic2.flac',) ('data/American/p300/p300_240_mic2.flac',) ('data/American/p299/p299_121_mic2.flac',) ('data/Irish/p298/p298_225_mic1.flac',) ('p300',) ('p299',) ('p300',) ('American',) ('Irish',)
('data/Irish/p295/p295_088_mic1.flac',) ('data/Irish/p295/p295_312_mic2.flac',) ('data/Irish/p283/p283_412_mic1.flac',) ('data/Scottish/p249/p249_231_mic1.flac',) ('p295',) ('p283',) ('p295',) ('Irish',) ('Scottish',)
('data/Scottish/p234/p234_173_mic1.flac',) ('data/Scottish/p234/p234_267_mic1.flac',) ('data/Scottish/p284/p284_304_mic2.flac',) ('data/American/p300/p300_006_mic2.flac',) ('p234',) ('p284',) ('p234',) ('Scottish',) ('American',)
('data/Irish/p283/p283_374_mic1.flac',) ('data/Irish/p283/p283_248_mic2.flac',) ('data/Irish/p295/p295_156_mic1.flac',) ('data/American/p299/p299_205_mic2.flac',) ('p283',) ('p295',) ('p283',) ('Irish',) ('American',)
('data/Irish/p298/p298_168_mic2.flac',) ('data/Irish/p298/p298_350_mic2.flac',) ('data/Irish/p245/p245_143_mic

('data/Irish/p245/p245_341_mic1.flac',) ('data/Irish/p245/p245_286_mic1.flac',) ('data/Irish/p283/p283_299_mic1.flac',) ('data/Indian/p376/p376_336_mic1.flac',) ('p245',) ('p283',) ('p245',) ('Irish',) ('Indian',)
('data/Irish/p283/p283_302_mic1.flac',) ('data/Irish/p283/p283_218_mic1.flac',) ('data/Irish/p245/p245_121_mic2.flac',) ('data/American/p299/p299_044_mic2.flac',) ('p283',) ('p245',) ('p283',) ('Irish',) ('American',)
('data/American/p301/p301_271_mic2.flac',) ('data/American/p301/p301_397_mic1.flac',) ('data/American/p299/p299_345_mic2.flac',) ('data/Irish/p283/p283_196_mic2.flac',) ('p301',) ('p299',) ('p301',) ('American',) ('Irish',)
('data/American/p300/p300_257_mic1.flac',) ('data/American/p300/p300_217_mic2.flac',) ('data/American/p301/p301_100_mic2.flac',) ('data/Irish/p245/p245_005_mic1.flac',) ('p300',) ('p301',) ('p300',) ('American',) ('Irish',)
('data/Indian/p251/p251_143_mic1.flac',) ('data/Indian/p251/p251_224_mic1.flac',) ('data/Indian/p248/p248_133_mic2.flac'

('data/American/p301/p301_188_mic1.flac',) ('data/American/p301/p301_188_mic2.flac',) ('data/American/p300/p300_141_mic2.flac',) ('data/Scottish/p249/p249_332_mic2.flac',) ('p301',) ('p300',) ('p301',) ('American',) ('Scottish',)
('data/Indian/p248/p248_285_mic2.flac',) ('data/Indian/p248/p248_085_mic1.flac',) ('data/Indian/p376/p376_214_mic2.flac',) ('data/Irish/p283/p283_247_mic1.flac',) ('p248',) ('p376',) ('p248',) ('Indian',) ('Irish',)
('data/American/p301/p301_079_mic2.flac',) ('data/American/p301/p301_291_mic1.flac',) ('data/American/p299/p299_256_mic1.flac',) ('data/Indian/p376/p376_203_mic1.flac',) ('p301',) ('p299',) ('p301',) ('American',) ('Indian',)
('data/Irish/p283/p283_103_mic1.flac',) ('data/Irish/p283/p283_099_mic1.flac',) ('data/Irish/p245/p245_041_mic1.flac',) ('data/Indian/p248/p248_006_mic1.flac',) ('p283',) ('p245',) ('p283',) ('Irish',) ('Indian',)
('data/Indian/p376/p376_071_mic2.flac',) ('data/Indian/p376/p376_141_mic2.flac',) ('data/Indian/p248/p248_003_mic2

('data/Scottish/p234/p234_205_mic1.flac',) ('data/Scottish/p234/p234_018_mic1.flac',) ('data/Scottish/p237/p237_093_mic1.flac',) ('data/Irish/p245/p245_266_mic1.flac',) ('p234',) ('p237',) ('p234',) ('Scottish',) ('Irish',)
('data/American/p300/p300_097_mic1.flac',) ('data/American/p300/p300_277_mic1.flac',) ('data/American/p301/p301_055_mic1.flac',) ('data/Indian/p248/p248_275_mic2.flac',) ('p300',) ('p301',) ('p300',) ('American',) ('Indian',)
('data/American/p301/p301_058_mic2.flac',) ('data/American/p301/p301_042_mic1.flac',) ('data/American/p299/p299_171_mic1.flac',) ('data/Indian/p248/p248_023_mic2.flac',) ('p301',) ('p299',) ('p301',) ('American',) ('Indian',)
('data/Irish/p245/p245_342_mic1.flac',) ('data/Irish/p245/p245_314_mic2.flac',) ('data/Irish/p295/p295_236_mic2.flac',) ('data/Indian/p376/p376_030_mic2.flac',) ('p245',) ('p295',) ('p245',) ('Irish',) ('Indian',)
('data/Indian/p251/p251_168_mic1.flac',) ('data/Indian/p251/p251_282_mic1.flac',) ('data/Indian/p376/p376_253_

KeyboardInterrupt: 

In [13]:
speaker_positive_id[0], speaker_negative_id, speaker_anchor_id, positive_accent, negative_accent

('p284', ('p237',), ('p284',), ('Scottish',), ('American',))

In [14]:
anchor_utter_path, positive_utter_path, negative_utter_path, accent_negative_path

(('data/Scottish/p284/p284_096_mic1.flac',),
 ('data/Scottish/p284/p284_161_mic2.flac',),
 ('data/Scottish/p237/p237_232_mic2.flac',),
 ('data/American/p300/p300_322_mic1.flac',))

In [21]:
class PairwiseDistance(Function):
    def __init__(self, p):
        super(PairwiseDistance, self).__init__()
        self.norm = p

    def forward(self, x1, x2):
        
        #print("above assert: ",x1, x2)
        assert x1.size() == x2.size()
        eps = 1e-4 / x1.size(1)
        diff = torch.abs(x1 - x2)
        out = torch.pow(diff, self.norm).sum(dim=1)
        return torch.pow(out + eps, 1. / self.norm)

class TripletLoss(Function):
    """
    Triplet loss function.
    """
    def __init__(self, margin=0.2):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.pdist = PairwiseDistance(2)  # norm 2

    def forward(self, anchor, positive, negative):
        d_p = self.pdist.forward(torch.from_numpy(anchor), positive)
        d_n = self.pdist.forward(torch.from_numpy(anchor), torch.from_numpy(negative))

        dist_hinge = torch.clamp(self.margin + d_p - d_n, min=0.0)
        loss = torch.mean(dist_hinge)
        return loss

In [22]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.inp1 = nn.Linear(in_features=256, out_features=100)
        self.inp2 = nn.Linear(in_features=100, out_features=100)
        self.b_n = nn.BatchNorm1d(128)
        
        self.drop_out = nn.Dropout(0.25)
        self.l1 = nn.Linear(200, 128)
        self.l2 = nn.Linear(128,128)
        self.out_acc = nn.Linear(128, 100)
        self.out_speak = nn.Linear(128,256)
        
    def forward(self, speaker_emb, acc_emb):
        x1 = F.relu(self.inp1(speaker_emb))
        
        x2 = F.relu(self.inp2(x1))
        #print(x1.shape, x2.shape, speaker_emb.shape, acc_emb.shape)
        out = torch.cat((x1, x2), axis=1)
        out = self.l1(out)
        out = F.relu(out)
        out = F.relu(self.l2(out))
        
        mf_acc = self.out_acc(out)
        mf_sp = self.out_speak(out)
        return F.relu(mf_sp), F.relu(mf_acc)

In [27]:
def train():
    device = hp.device
    dataset = TripletSpeakerDataset()
    data_loader = torch.utils.data.DataLoader(dataset, drop_last=True)
    ae = AutoEncoder().to('cuda')
    writer = SummaryWriter('triplet_loss_logs')
    optimizer = optim.Adam(ae.parameters())
    optimizer.zero_grad()
    #ae.load_state_dict(torch.load(''))
    trip_loss = TripletLoss()
    total_loss = 0
    for epoch in range(20):
        for batch_id, (anchor_utter_path, positive_utter_path, negative_utter_path, accent_negative_path, speaker_positive_id, speaker_negative_id, speaker_anchor_id, positive_accent, negative_accent) in tqdm(enumerate(data_loader), total = len(data_loader)):
            anchor_utter_path, positive_utter_path, negative_utter_path = anchor_utter_path[0], positive_utter_path[0], negative_utter_path[0]
            accent_negative_path, speaker_positive_id, speaker_negative_id = accent_negative_path[0], speaker_positive_id[0], speaker_negative_id[0]
            speaker_anchor_id, positive_accent, negative_accent = speaker_anchor_id[0], positive_accent[0], negative_accent[0]
            
            try:
                pre_positive_speaker_embs, pre_positive_accent_embs = get_both_embs(positive_utter_path)
                pre_positive_speaker_embs, pre_positive_accent_embs = Variable(torch.from_numpy(pre_positive_speaker_embs)).to('cuda'), Variable(torch.from_numpy(pre_positive_accent_embs)).to('cuda')
                anchor_utter_path
                anchor_speaker_embs, anchor_accent_embs = get_both_embs(anchor_utter_path)
                negative_speaker_embs, _ = get_both_embs(negative_utter_path)
                _, negative_accent_embs = get_both_embs(accent_negative_path)
            except ValueError:
                print("ValueError")
                continue
            except:
                continue
            positive_speaker_embs, positive_accent_embs = ae.forward(pre_positive_speaker_embs, pre_positive_accent_embs)
            positive_speaker_embs, positive_accent_embs = positive_speaker_embs.to('cpu'), positive_accent_embs.to('cpu')
            optimizer.zero_grad()
            #print(anchor_accent_embs.shape)
            accent_trip_loss = trip_loss.forward(anchor_accent_embs, positive_accent_embs, negative_accent_embs)
            speaker_trip_loss = trip_loss.forward(anchor_speaker_embs, positive_speaker_embs, negative_speaker_embs)
            criterion = accent_trip_loss + speaker_trip_loss
            total_loss += criterion
            
            
            
            criterion.backward()
            optimizer.step()
            writer.add_scalar('Loss', criterion, epoch*len(data_loader)+1)
            
            if (batch_id + 1) % 5 == 0:
                mesg = "{}\tEpoch:{},Iteration:{}\tLoss:{}\t\n".format(time.ctime(), epoch+1,
                        batch_id+1, criterion)
                print(mesg)
                if hp.train.log_file is not None:
                    with open(hp.train.log_file,'a') as f:
                        f.write(mesg)
            if hp.train.checkpoint_dir is not None and (epoch + 1) % hp.train.checkpoint_interval == 0:
                embedder_net.eval()
                ckpt_model_filename = "ckpt_epoch_" + str(epoch+1) + "_batch_id_" + str(batch_id+1) + ".pth"
                ckpt_model_path = os.path.join("new_checkpoints/", ckpt_model_filename)
                torch.save(embedder_net.state_dict(), ckpt_model_path)
                embedder_net.to(device).train()
        embedder_net.eval()
        save_model_filename = "final_epoch_" + str(epoch + 1) + "_batch_id_" + str(batch_id + 1) + ".model"
        save_model_path = os.path.join(hp.train.checkpoint_dir, save_model_filename)
        torch.save(embedder_net.state_dict(), save_model_path)

        print("\nDone, trained model saved at", save_model_path)

In [ ]:
train()




  0%|          | 0/10956 [00:00<?, ?it/s]

(1, 40, 180)
(1, 40, 180)
(1, 40, 180)





  0%|          | 1/10956 [00:01<4:15:07,  1.40s/it]

(1, 40, 180)
(1, 40, 180)
(1, 40, 180)
(1, 40, 180)





  0%|          | 2/10956 [00:02<4:04:57,  1.34s/it]

...

(1, 40, 180)
(1, 40, 180)
(1, 40, 180)
(1, 40, 180)





  1%|          | 83/10956 [01:35<3:22:09,  1.12s/it]

(1, 40, 180)
(1, 40, 180)
(1, 40, 180)
(1, 40, 180)





  1%|          | 84/10956 [01:36<3:14:17,  1.07s/it]

(1, 40, 180)


In [ ]:
torch.transpose

In [64]:
len(data_loader)

10956

In [49]:
arr = np.array([[[1,2],[2,4],[4,5]],[[1,2],[2,4],[4,5]],[[1,2],[2,4],[4,5]],[[1,2],[2,4],[4,5]]])
arr.shape

(4, 3, 2)

In [59]:
np.transpose(arr, axes=(0,2,1))

array([[[1, 2, 4],
        [2, 4, 5]],

       [[1, 2, 4],
        [2, 4, 5]],

       [[1, 2, 4],
        [2, 4, 5]],

       [[1, 2, 4],
        [2, 4, 5]]])

In [60]:
torch.transpose(torch.Tensor(arr), dim0=1, dim1=2).detach().numpy()

TypeError: numpy() takes no arguments (1 given)

In [28]:
positive_speaker_embs, positive_accent_embs = get_both_embs(positive_utter_path)

start text independent utterance feature extraction



In [34]:
positive_speaker_embs.shape

torch.Size([1, 256])

In [32]:
positive_accent_embs.shape

(1, 100)

In [35]:
negative_speaker_embs, _ = get_both_embs(negative_utter_path)

start text independent utterance feature extraction


In [ ]:
import numpy as np
from sklearn.manifold import TSNE

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
tsne = TSNE(verbose=1)

In [ ]:
res = tsne.fit_transform(utters, y = speakers)

In [ ]:
for ele in utts:
    print(ele)

In [ ]:
np_utts = np.asarray(utts, dtype='float32')

In [ ]:
utts = np.load('utterances.npy', allow_pickle=True)
speakers = np.load('speakers.npy', allow_pickle=True)

In [ ]:
embedder_net(j[:,0,...].to('cuda'))

In [ ]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
y = np.asarray(speakers)
X = np.squeeze(np.asarray(utters),1)
print(X.shape, y.shape)

In [ ]:
feat_cols = [ 'pixel'+str(i) for i in range(X.shape[1]) ]
df = pd.DataFrame(X,columns=feat_cols)
df['y'] = y
df['label'] = df['y'].apply(lambda i: str(i))
X, y = None, None
print('Size of the dataframe: {}'.format(df.shape))

In [ ]:
# For reproducability of the results
np.random.seed(42)
rndperm = np.random.permutation(df.shape[0])

In [ ]:
N = 10000
df_subset = df.loc[rndperm[:N],:].copy()
data_subset = df_subset[feat_cols].values
pca = PCA(n_components=3)
pca_result = pca.fit_transform(data_subset)
df_subset['pca-one'] = pca_result[:,0]
df_subset['pca-two'] = pca_result[:,1] 
df_subset['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
np.squeeze(utters, axis=1).shape

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=10000)
tsne_results = tsne.fit_transform(pd.DataFrame(np.squeeze(np.asarray(utters),1)))
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
df_subset['tsne-2d-one'] = tsne_results[:,0]
df_subset['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 168),
    data=df_subset,
    legend="brief",
    alpha=0.3
)

In [ ]:
y

In [ ]:
plt.scatter(tsne_results[:,0], tsne_results[:,1])

In [ ]:
sns.lmplot(x='_wef_',
          y='__ewwf',
          data = tsne_results,
          fit_reg=False,
          legend=True,
          hue='speakers')

In [ ]:
np.unique(y).shape